In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import warnings
warnings.filterwarnings("ignore")
from numba import njit, prange

## Function definition

In [ ]:
# Speed function
def V(x): 
    if isinstance(x,list):
        return [np.square(1-i/4500)*9.78 for i in x]
    else:
        return np.square(1-x/4500)*9.78

# Event-based simulation, reffering to Algorithm 1 in the paper
def within_day_process(Departure_time,trip_len,Number_of_user,k):
    Arrival_time=Departure_time+trip_len/9.78/60
    Event_list1=pd.DataFrame()
    Event_list1['time(min)']=np.array(Departure_time)
    Event_list1['event']=np.array(['departure']*Number_of_user)
    Event_list1['trip_len(m)']=trip_len
    Event_list2=pd.DataFrame()
    Event_list2['time(min)']=np.array(Arrival_time)
    Event_list2['event']=np.array(['arrival']*Number_of_user)
    Event_list2['trip_len(m)']=trip_len
    Event_list = pd.concat([Event_list1,Event_list2])
    S_Event_list=Event_list.sort_values(by = ['time(min)'],ascending = True)
    S_Event_list['veh_index'] = S_Event_list.index
    n=0 # Number of vehicle (accumulation)
    j=0 # index of event
    vehicle_information=pd.DataFrame({'vehicle':range(Number_of_user),'trip_len(m)':trip_len,'t_exp':np.zeros(Number_of_user)\
                                     ,'account':np.zeros(Number_of_user)})
    t=[] # event time
    t.append(S_Event_list['time(min)'].iloc[0]) #initial time
    vehicle_index=[]
    Accumulation=[]
    Speed=[]
    vehicle_information['account']=I-custgauss(Departure_time,**params)*Event_list1['trip_len(m)']*k
    while S_Event_list.empty==False:
        j=j+1
        t.append(S_Event_list['time(min)'].iloc[0])
        if S_Event_list['event'].iloc[0]=='departure':     
            vehicle_index.append(S_Event_list[(S_Event_list['time(min)']==t[j])].index.values[0])

            trip_len1=vehicle_information['trip_len(m)'].to_numpy()
            trip_len1[vehicle_index[0:-1]]=trip_len1[vehicle_index[0:-1]]-V(n)*60*(t[j]-t[j-1])
            vehicle_information['trip_len(m)']=trip_len1

            n=n+1
            Accumulation.append(n)
            vehicle_information.loc[vehicle_index[-1],'t_ins']=trip_len1[vehicle_index[-1]]/V(n)/60
            

            S_Event_list.loc[(S_Event_list['event']=='arrival') & (S_Event_list['veh_index'].isin(vehicle_index[0:-1])),'time(min)']=\
            t[j]+vehicle_information.loc[vehicle_index[0:-1],'trip_len(m)']/V(n)/60

            S_Event_list.loc[(S_Event_list['event']=='arrival') & (S_Event_list['veh_index'].isin(list(set(S_Event_list['veh_index'].values).difference(set(vehicle_index[0:-1]))))),'time(min)']=\
            Event_list1['time(min)'].values[list(set(S_Event_list['veh_index'].values).difference(set(vehicle_index[0:-1])))]+\
            vehicle_information.loc[list(set(S_Event_list['veh_index'].values).difference(set(vehicle_index[0:-1]))),'trip_len(m)']/V(n)/60

        else:
            trip_len1=vehicle_information['trip_len(m)'].values
            trip_len1[vehicle_index]=trip_len1[vehicle_index]-V(n)*60*(t[j]-t[j-1])
            vehicle_information['trip_len(m)']=trip_len1

            n=n-1
            Accumulation.append(n)

            vehicle_information.at[S_Event_list['veh_index'].iloc[0],'t_exp']=\
            S_Event_list.at[S_Event_list['veh_index'].iloc[0],'time(min)']-\
            Departure_time[S_Event_list['veh_index'].iloc[0]]
    #         Event_list1.loc[S_Event_list['veh_index'].iloc[0],'time(min)']

            vehicle_index.remove(S_Event_list[0:1].index.values[0])
        
            S_Event_list.loc[(S_Event_list['event']=='arrival') & (S_Event_list['veh_index'].isin(vehicle_index)),'time(min)']=\
            t[j]+vehicle_information.loc[vehicle_index,'trip_len(m)']/V(n)/60

            S_Event_list.loc[(S_Event_list['event']=='arrival') & (S_Event_list['veh_index'].isin(list(set(range(Number_of_user)).difference(set(vehicle_index))))),'time(min)']=\
            Event_list1.loc[list(set(range(Number_of_user)).difference(set(vehicle_index)))]['time(min)']+\
            vehicle_information.loc[list(set(range(Number_of_user)).difference(set(vehicle_index))),'trip_len(m)']/V(n)/60

        S_Event_list = S_Event_list.iloc[1:]    
        S_Event_list=S_Event_list.sort_values(by = ['time(min)'],ascending = True)
        Speed.append(V(n))
    vehicle_information['t_dep']=Event_list1['time(min)']
    vehicle_information['t_arr']=vehicle_information['t_dep']+vehicle_information['t_exp']
    time_list=pd.concat([vehicle_information['t_dep'],vehicle_information['t_arr']])
    time_list=time_list.sort_values(ascending = True)
    return vehicle_information, time_list, Accumulation, Speed


def rearrange(df, cols):
    
    all_values=[]
    for idx,i in enumerate(cols):
        
        vals=df[i].T[idx].values
        all_values.append(vals)
    
    if len(cols[0])>1:
        return pd.DataFrame(data=np.vstack(all_values),columns=('t'+str(i) for i in range(2*tao+1)))
    else:
        return pd.DataFrame(data=np.vstack(all_values))
    

def schedule_delay(T_diff):
    SD=np.empty((Number_of_user,2*tao+1))
    for i in range(2*tao+1):
        SD[:,i]=L*T_diff[:,i]*(1-np.array(T_diff[:,i]<0).astype(int))-\
        E*T_diff[:,i]*np.array(T_diff[:,i]<0).astype(int)
    return SD

@njit
def column_generate(time_slot,case='all'):
    columns=[]
    if case=='all':
        for i in range(Number_of_user):
            columns.append([j for j in range(np.sum(time_slot[i]<Departure_time[i])-tao,\
                                                      np.sum(time_slot[i]<Departure_time[i])+tao+1)])
    elif case=='chosen':
        for i in range(Number_of_user):
            columns.append([np.sum(time_slot[i]<Departure_time[i])])
    return columns


@njit
def V1(x):
    return np.square(1-x/4500)*9.78


# travel time experienced by fictional users
@njit
def fic_tt(user, time_point, all_time_matrix, new_timelist, trip_len, Accumulation,Number_of_user):
    star_time=all_time_matrix[user,time_point]
    known_list=new_timelist[new_timelist>star_time]
    if len(known_list)==0: # depart after all real travelers leave
        texp=trip_len[user]/9.78/60
    elif len(known_list)==Number_of_user*2: # depart before all real travelers start
        texp=0
        count=0
        left_len=trip_len[user]-9.78*60*(known_list[0]-star_time)
        if left_len<0:
            texp=trip_len[user]/9.78/60
        else:
            V_list=np.array([V1(x) for x in Accumulation[Number_of_user*2-len(known_list):-1]])
            len_piece=np.diff(known_list)*V_list*60
            cum_len=np.cumsum(len_piece)
            count=np.sum(cum_len<left_len)
            texp=known_list[count+1]-star_time+(left_len-cum_len[count])/V1(Accumulation[count])/60
    else:
        texp=0
        count=0
        left_len=trip_len[user]-V1(Accumulation[Number_of_user*2-len(known_list)-1])*(known_list[0]-star_time)*60
        if left_len<0:
            texp=trip_len[user]/V1(Accumulation[Number_of_user*2-len(known_list)-1])/60
        else:
            V_list=np.array([V1(x) for x in Accumulation[Number_of_user*2-len(known_list):-1]])
            len_piece=np.diff(known_list)*V_list*60
            cum_len=np.cumsum(len_piece)
            count=np.sum(cum_len<left_len)
            if count==len(cum_len):
                texp=known_list[count]-star_time+(left_len-cum_len[count-1])/9.78/60
            else:
                texp=known_list[count+1]-star_time+(left_len-cum_len[count])/V1(Accumulation[Number_of_user*2-len(known_list)+count])/60
    return texp


@njit(parallel=True)
def T_est(all_time_matrix, new_timelist, trip_len, Accumulation, Number_of_user, tao):
    T_estimate_array=np.zeros((Number_of_user,2*tao+1))
    for i in prange(Number_of_user):
        for j in prange(2*tao+1):
            T_estimate_array[i,j]=fic_tt(i, j, all_time_matrix, new_timelist, trip_len, Accumulation,Number_of_user)
    return T_estimate_array

## Parameter settings

In [ ]:
np.random.seed(seed=59)
tao=30 # number of time intervals
Number_of_user=3000
Departure_time=np.random.normal(80,18,Number_of_user)
for i in range(len(Departure_time)):
    if Departure_time[i]<20 or Departure_time[i]>150:
        Departure_time[i]=np.random.normal(80,18,1)
plt.hist(Departure_time,bins=20)

In [ ]:
trip_len=np.array(np.zeros(Number_of_user))
E=np.array(np.zeros(Number_of_user)) # schedule delay early penalty
L=np.array(np.zeros(Number_of_user)) # schedule delay late penalty

for i in range(Number_of_user):
    trip_len[i]=4600+np.random.normal(0,(0.02*4600)**2)
    while trip_len[i]<20:
        trip_len[i]=4600+np.random.normal(0,(0.02*4600)**2)
        
Wished_Arrival_time=Departure_time+trip_len/9.78/60 # wished arrival time

for i in range(Number_of_user):
    E[i]=0.5+np.random.normal(0.05**2,0.1**2)
    L[i]=4+np.random.normal(0.1**2,0.4**2)
    while E[i]<0.3 or E[i]>0.7:
        E[i]=0.5+np.random.normal(0.05**2,0.1**2)
    while L[i]<2.5 or L[i]>5.5:
        L[i]=4+np.random.normal(0.1**2,0.4**2)
        
Dep_time_set=pd.DataFrame()
for i in range(2*tao+1):
    Dep_time_set['t'+str(i)]=Departure_time-tao+i

In [ ]:
I=5 #initial endowment for every traveller everyday
params={'A': 11,
  'mu': 80,
  'sigma': 18}
x=np.linspace(-10,130,500)

# toll profile
def custgauss(x,mu,sigma,A):
    return A*np.exp(-(x-mu)**2/2/sigma**2)
toll_profile=custgauss(x,**params)

In [ ]:
Sim_day=50
omega=0.7
norm_list=[]
Acc_df=pd.DataFrame()
time_label=pd.DataFrame()
all_time_slot=pd.DataFrame()
cost_list=[]
cs_list=[]
price=np.ones(1)*0 # initial price as 0
b=0.0002 #price adjustment parameter 
alpha=1.1 # value of time
k=0.0002 # trip length scale factor
account_state=[]
tt=[]
Eachday_dep=pd.DataFrame()
Eachday_dep['d0']=Departure_time

for i in range(2*tao+1):
    all_time_slot['t'+str(i)]=Departure_time-tao+i

acco=[]
all_time_matrix=np.array(all_time_slot)
utility={}

# choose a case to simulate
Base_Case=False
TCS_Case=True
CP_Case=False

## Main simulation

In [ ]:
for d in range(Sim_day):
    start=time.time()
    vehicle_information, time_list, Accumulation, Speed=within_day_process(Departure_time,trip_len,Number_of_user)

    Acc_df['d'+str(d)]=Accumulation
    time_label['d'+str(d)]=time_list.tolist()
    vehicle_information['origin_tl']=trip_len

    account_state.append(vehicle_information['account'])
    tt.append(vehicle_information['t_exp'])
    
    new_timelist=np.array(time_list)
    
    T_estimate=T_est(all_time_matrix, new_timelist, trip_len, Accumulation, Number_of_user, tao)
    T_estimated=pd.DataFrame(T_estimate,columns=('t'+str(i) for i in range(2*tao+1)))

    T_estimated_diff=pd.DataFrame()
    for j in range(2*tao+1):
        T_estimated_diff['t'+str(j)]=T_estimated['t'+str(j)]+all_time_slot['t'+str(j)]-Wished_Arrival_time

    c_estimated=pd.DataFrame(columns=('t'+str(i) for i in range(2*tao+1)))
    c_cs=pd.DataFrame(columns=('t'+str(i) for i in range(2*tao+1)))
    T_diff=np.array(T_estimated_diff)
    SD=schedule_delay(T_diff)
    for j in range(2*tao+1):
        c_estimated['t'+str(j)]=alpha*(T_estimated['t'+str(j)]+SD[:,j])+\
        price[d]*custgauss(all_time_slot['t'+str(j)],**params)*np.array(vehicle_information['origin_tl'])*0.0002

        c_cs['t'+str(j)]=alpha*(T_estimated['t'+str(j)]+SD[:,j])

    if d==0:
        c_perceived=c_estimated
    else:
        c_perceived=omega*c_perceived+(1-omega)*c_estimated
    norm_list.append(np.linalg.norm(c_perceived-c_estimated,ord=1)/Number_of_user)
    theta=5*10**(-1) #scale parameter

    acco.append(sum(account_state[d]))
    if Base_Case:
        price=np.append(price,0)
    elif TCS_Case:
        price=np.append(price,max(price[d]-b*acco[d],0))
    elif CP_Case:
        price=np.append(price,1)
    
    window_c_perceived=c_perceived
    
    columns1=[]
    for i in range(Number_of_user):
        columns1.append(['t'+str(np.sum(all_time_slot.iloc[i].values<Departure_time[i]))])
    window_c_exp=rearrange(df=c_estimated,cols=columns1)
    window_cs=rearrange(df=c_cs,cols=columns1)

    cost_list.append(window_c_exp.sum()) # generalized cost: tt cost + sd cost + credit payment (or pricing payment)
    cs_list.append(window_cs.sum()) # travel cost: tt cost + sd cost
    
    util_rand = np.random.gumbel(-0.57721 / theta, 1.0 / theta, (Number_of_user,2*tao+1))
    utility_exp=-window_c_perceived+util_rand
    Departure_time=np.diag(Dep_time_set[utility_exp.idxmax(axis=1)])
    utility['d'+str(d)]=np.diag(utility_exp[utility_exp.idxmax(axis=1)])+np.diag(window_c_perceived[utility_exp.idxmax(axis=1)])
    
    Eachday_dep['d'+str(d+1)]=Departure_time
    end = time.time()
    print('day',d,':',end-start,cost_list[d][0],cs_list[d][0],price[d])

## Output

### Computation of generalized cost, travel cost, consumer surplus, social welfare and random utility

In [ ]:
gc=np.zeros(Sim_day) # generalized cost
tc=np.zeros(Sim_day) # travel cost
for i in range(Sim_day):
    gc[i]=cost_list[i][0]
    tc[i]=cs_list[i][0]
    
csu=np.zeros(Sim_day) # consumer surplus
sw=np.zeros(Sim_day) # social welfare
ru=np.zeros(Sim_day) # random utility
for i in range(Sim_day):
    csu[i]=-gc[i]+np.sum(utility['d'+str(i)])
    ru[i]=np.sum(utility['d'+str(i)])
    sw[i]=-tc[i]+np.sum(utility['d'+str(i)])
    
ttt=np.zeros(Sim_day) # travel time
for i in range(Sim_day):
    ttt[i]=sum(tt[i])

### Number of bought and sold credits

In [ ]:
buy=[]
sell=[]
for i in range(Sim_day):
    buy.append(-sum(account_state[i][account_state[i]<=0]))
    sell.append(sum(account_state[i][account_state[i]>0]))

### Visualization

#### Evolution of inconsistency

In [ ]:
plt.plot(range(1,Sim_day),norm_list[1:],color='red')
plt.xlabel('Days')
plt.ylabel('$|C_d-c_d|_1$/$N$ [DKK]')

#### Evolution of consumer surplus

In [ ]:
plt.plot(range(1,Sim_day),csu[1:]/Number_of_user,color='red')
ax2.set_xlabel('Days')
ax2.set_ylabel('Average consumer surplus [DKK]')

#### Evolution of social welfare

In [ ]:
plt.plot(range(1,Sim_day),sw[1:]/Number_of_user,color='red')
plt.xlabel('Day')
plt.ylabel('$W_{TCS}$ [DKK]')

#### Evolution of accumulation

In [ ]:
x=np.linspace(0,180,500) # ploting scale
toll_profile=custgauss(x,**params)
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
# a4,=ax2.plot(x,toll_profile,linestyle='--',color='gray') # uncommented when ploting the toll profile together
a1,=ax1.plot(time_label['d0'],Acc_df['d0'], color='red')
# a2,=ax1.plot(time_label['d10'],Acc_df['d10'], color='orange') # uncommented when ploting accumulation curve of another day
a3,=ax1.plot(time_label['d35'],Acc_df['d35'], color='green')
a5,=ax1.plot(time_label['d49'],Acc_df['d49'], color='blue')

# plt.legend([a1,a3,a5,a4], ['day 0','day 35','day 49','Toll profile'])
plt.legend([a1,a3,a5,a4], ['day 0','day 35','day 49'])
ax1.set_xlabel('Time [min]')
ax1.set_ylabel('Accumulation [traveler]')
# ax2.set_ylabel('Credit charge',color='gray')

#### Evolution of buying and selling

In [ ]:
plt.plot(range(Sim_day),buy[0:Sim_day],color='red')
plt.plot(range(Sim_day),sell[0:Sim_day],color='green')
plt.xlabel('Days')
plt.ylabel('Number of credits')
plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
plt.legend(['Number of bought credits','Number of sold credits']) 

#### Evolution of credit price

In [ ]:
plt.plot(range(Sim_day+1),price,color='red')
plt.xlabel('Days')
plt.ylabel('Credit price [DKK]')